In [ ]:
import anndata  as ad  
from pycisTopic.cistopic_class import create_cistopic_object 

import pycisTopic  
print(pycisTopic.__version__)  

import scenicplus
scenicplus.__version__

import scanpy as sc
import sys
#import ostemp_dir
import pickle

import anndata  as ad  
from pycisTopic.cistopic_class import create_cistopic_object  
import numpy as np  
import pandas as pd 
#import celltypist
import os

work_dir = '/media/AnalysisDisk2/hIV/all'
out_dir =work_dir
tmp_dir = '/media/AnalysisDisk2/Temp'
Topic_dir = "/media/AnalysisDisk2/hIV/all/n_iter200"
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
if not os.path.exists(Topic_dir):
    os.makedirs(Topic_dir)

atac_all=ad.read_h5ad('/media/AnalysisDisk2/hIV/SEAcell-ATAC-RNA/ATAC/Final_L3_SEAcell.h5ad')
atac_all

cell_barcodes = atac_all_subset.obs_names  
peaks = atac_all_subset.var_names  

cistopic_obj = create_cistopic_object(  
    fragment_matrix=atac_all_subset.X.T.tocsr(),  
    cell_names=cell_barcodes.to_list(),  
    region_names=peaks.to_list(),  
    tag_cells = False,
    split_pattern = '_' 
)  
print(cistopic_obj)

cistopic_obj.add_cell_data(atac_all_subset.obs)


pickle.dump(
    cistopic_obj,
    open(os.path.join( "cistopic_obj_nomodels.pkl"), "wb")
)

%%time
tmp_dir = '/media/AnalysisDisk2/Chenjunjie/Temp'
os.environ['MALLET_MEMORY'] = '500G'
from pycisTopic.lda_models import run_cgs_models_mallet
# Configure path Mallet
mallet_path="/media/AnalysisDisk2/hIV/04scenicplus-test/Mallet-202108/bin/mallet"
# Run models
models=run_cgs_models_mallet(
    cistopic_obj,
    n_topics=[ 5, 10, 20, 30, 40, 50,60,70,80],
    n_cpu=80,
    n_iter=200,
    random_state=555,
    alpha=50,
    alpha_by_topic=True,
    eta=0.1,
    eta_by_topic=False,
    tmp_path=tmp_dir,
    save_path= Topic_dir,
    mallet_path=mallet_path,
)

topic_model_dir =os .listdir(Topic_dir)
models = []
for file in topic_model_dir:
    file_path = os.path.join(Topic_dir, file)
    topic_temp = pickle.load(open(file_path, 'rb'))
    models.append(topic_temp)
    print(file)

from pycisTopic.lda_models import *
model = evaluate_models(models,
                       select_model=40,
                       return_model=True,
                       plot_metrics=False)

from pycisTopic.lda_models import *
model = evaluate_models(models,
                       select_model=50,
                       return_model=True,
                       plot_metrics=False)

cistopic_obj.add_LDA_model(model)

pickle.dump(
    cistopic_obj,
    open(os.path.join( "cistopic_obj.pkl"), "wb")
)

from pycisTopic.clust_vis import run_umap
run_umap(
    cistopic_obj,
    target  = 'cell', scale=True)

plot_metadata(  
    cistopic_obj,  
    reduction_name='UMAP',  
    variables=['people'],  
    show_label=False, 
    dot_size=0.5,  
    figsize=(8, 8)  
)  

from pycisTopic.clust_vis import plot_metadata
plot_metadata(
    cistopic_obj,
    dot_size = 0.5,
    reduction_name = 'UMAP',
    #color_dictionary = {'final_annotation': L2_annotation_colors},
    variables = ['celltype_L3'],
    figsize = (10, 10))

pickle.dump(
    cistopic_obj,
    open(os.path.join( "cistopic_obj.pkl"), "wb")
)

atac_all_subset.obs['celltype_L3_stage'].value_counts()

from pycisTopic.clust_vis import plot_topic
plot_topic(cistopic_obj, reduction_name = 'UMAP', num_columns = 5)

cistopic_obj.cell_data['celltype_str'] = cistopic_obj.cell_data['celltype_L3_stage'].astype(str)

from pycisTopic.clust_vis import *
cell_topic_heatmap(cistopic_obj,  
                   variables=['celltype_str'],  
                   #color_dict=final_annotation_colors,
                   scale=False,  
                   legend_loc_x=1.05,  
                   legend_loc_y=-1.2,  
                   legend_dist_y=-1,  
                   figsize=(10, 15),  
                   color_dictionary={})  

from pycisTopic.topic_binarization import binarize_topics
region_bin_topics_top_3k = binarize_topics(
    cistopic_obj, method='ntop', ntop = 3_000,
    plot=True, num_columns=5
)

region_bin_topics_otsu = binarize_topics(
    cistopic_obj, method='otsu',
    plot=True, num_columns=5
)

pickle.dump(
    cistopic_obj,
    open(os.path.join( "cistopic_obj.pkl"), "wb")
)

from pycisTopic.diff_features import *
imputed_acc_obj = impute_accessibility(cistopic_obj, selected_cells=None, selected_regions=None, scale_factor=10**6)
normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)
variable_regions = find_highly_variable_features(normalized_imputed_acc_obj, min_disp = 0.05,
    min_mean = 0.0125,
    max_mean = 3,
    max_disp = np.inf,
    n_bins=20,
    n_top_features=None,
    plot=True)

%time
markers_dict= find_diff_features(
    cistopic_obj,
    imputed_acc_obj,
    variable='celltype_L3_stage',
    var_features=variable_regions,
    contrasts=None,
    adjpval_thr=0.05,
    log2fc_thr=np.log2(1.2),
    n_cpu=90,
    _temp_dir= tmp_dir,
    split_pattern = None
)

print("Number of DARs found:")
print("---------------------")
for x in markers_dict:
    print(f"  {x}: {len(markers_dict[x])}")

os.makedirs(os.path.join(work_dir, "region_sets"), exist_ok = True)
os.makedirs(os.path.join(work_dir, "region_sets", "Topics_otsu"), exist_ok = True)
os.makedirs(os.path.join(work_dir, "region_sets", "Topics_top_3k"), exist_ok = True)
os.makedirs(os.path.join(work_dir, "region_sets", "DARs_cell_type"), exist_ok = True)

from pycisTopic.utils import region_names_to_coordinates

for topic in region_bin_topics_otsu:
    region_names_to_coordinates(
        region_bin_topics_otsu[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_otsu", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

for topic in region_bin_topics_top_3k:
    region_names_to_coordinates(
        region_bin_topics_top_3k[topic].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "Topics_top_3k", f"{topic}.bed"),
        sep = "\t",
        header = False, index = False
    )

for cell_type in markers_dict:
    region_names_to_coordinates(
        markers_dict[cell_type].index
    ).sort_values(
        ["Chromosome", "Start", "End"]
    ).to_csv(
        os.path.join(out_dir, "region_sets", "DARs_cell_type", f"{cell_type}.bed"),
        sep = "\t",
        header = False, index = False
    )

# Running Scenic＋

import os
os.chdir("/media/AnalysisDisk2/hIV/04scenicplus-test/scenicplus_p3/scplus_pipeline/Snakemake")

!cat /media/AnalysisDisk2/hIV/scenicplus_p3/scplus_pipeline/Snakemake/config/config.yaml

!snakemake --cores 60